In [2]:
import numpy as np


def troca_linha(a,i,j):
    a[[i,j]] = a[[j,i]]

def multiplica_linha(a,i,l):
    a[i] *= l

def soma_linha(a,i,j,l):
    a[i] += l*a[j]

def troca_coluna(a, i, j):
    a[:,[i,j]] = a[:,[j,i]]

def multiplica_coluna(a, i, l):
    a[:,i] *= l

def soma_coluna(a, i, j, l):
    a[:,i] += l*a[:,j]

def conta_colunas(A,m):
    _,n = A.shape

    cont = 0
    for j in range(n):
        coluna = A[:m,j]
        zero = False
        for valor in coluna:
            if np.round(valor,10) != 0:
                zero = True
                break
        if zero:
            cont += 1
    
    return cont

def bases_aumentadas(Matriz, m):

    zeros = conta_colunas(Matriz,m)
    Im = Matriz[:m,:zeros].T
    Im = completar_base(Im)
    Ker = Matriz[m:,:].T

    return Im,Ker

def completar_base(A):
    m,n = A.shape
    while m != n:
        linha_adicional = np.zeros((1,n), dtype = float)
        A = np.concatenate((A,linha_adicional), axis=0)
        m,n = A.shape
    for i in range(m):
        pivo = A[i,i]
        if pivo == 0:
            A[i,i] = 1
            base_nucleo = guardando_gauss_jordan(A,i)
            A[i] = base_nucleo
    # ... (código para encontrar soluções particulares)
    
    return A

def guardando_gauss_jordan(A,indice):
    _,n = A.shape
    valores = []
    for col in range(indice,n):
        for linha in range(col+1, n):
            soma_linha(A, col, linha, -A[linha,col])
            valores.append(-A[linha,col])
        for linha in range(col):
            valores.append(-A[linha,col])
        valores.append(A[linha,linha]) 
        return np.matrix(valores,dtype=float)

def eliminacao_gauss(A, tol=1e-10):
    m,n = A.shape
    I = np.eye(n,dtype = float)
    M = np.vstack((A,I))
    

    for linha in range(m):
        if np.round(np.abs(M[linha,linha]),10) <= tol:
            for col in range(linha+1, n):
                if np.round(np.abs(M[linha,col]),10) != 0:
                    troca_coluna(M, col, linha)
                    break
                    
        if np.round(np.abs(M[linha,linha]),10) > tol:
            multiplica_coluna(M, linha, 1/M[linha,linha])

        for col in range(linha+1, n):
            soma_coluna(M, col, linha, -M[linha,col])
        for col in range(linha):
            soma_coluna(M, col, linha, -M[linha,col])

    Im, Ker = bases_aumentadas(M, m)
    #Arredondando os valores das matrizes para 10 casas decimais
    Im, Ker = Im.round(10), Ker.round(10) 
    return Im,Ker

In [6]:
import numpy as np

def norma(u):
    soma = np.sum(u**2)
    modulo = np.sqrt(soma)
    return modulo

def normaliza_vet(v):
    modulo = norma(v)
    v /= modulo

def gram_Schmidt(v1,v2):
    p_i = (v1@v2)/(v2@v2)
    w = v1 - v2*p_i
    return w

def normalizar(M):
    m,_ = M.shape
    
    for linha in range(m):
        normaliza_vet(M[linha])

    return M

def ortogonalizar(M):
    m,n = M.shape
    V = np.zeros((m,n))

    for linha in range(m):
        V[linha] = M[linha]
        for col in range(linha-1,-1,-1):
            V[linha] = gram_Schmidt(V[linha], V[col])
    
    V = normalizar(V)
    V = np.matrix(V)
    return V.round(10)

In [7]:
def matriz_r(w1,u1):
    R = np.dot(w1,u1.I)

    return R.round(10)

def transformacao_ortogonal(w1,w2,u1,u2,mat_t):
    R1 = matriz_r(np.matrix(w1),np.matrix(u1))
    R2 = matriz_r(np.matrix(u2),np.matrix(w2))
    transformacao = np.dot(np.dot(R1,mat_t),R2)
    
    return transformacao.round(10)

In [8]:
B = np.matrix([[-0.17, -0.05, -0.46, -0.05, -0.39,  0.,   -0.01,  0.33, -0.38,  0.37, -0.23, -0.08],
 [ 0.15,  0.09, -0.12, -0.46, -0.34,  0.38, -0.08, -0.09, -0.13,  0.05,  0.17,  0.15],
 [-0.26,  0.04, -0.1,   0.22, -0.32,  0.03,  0.13,  0.27, -0.25,  0.43, -0.29, -0.04],
 [-0.45,  0.12,  0.54, -0.14,  0.26,  0.26, -0.14, -0.23,  0.24, -0.17, -0.22, -0.01],
 [-0.02,  0.03,  0.03, -0.52, -0.29,  0.47, -0.11, -0.08, -0.06, -0.06,  0.24,  0.17],
 [ 0.31,  0.57, -0.18, -0.3,  -0.35,  0.21, -0.15,  0.24,  0.04, -0.07, -0.25, -0.07],
 [-0.19,  0.24,  0.16, -0.31, -0.29,  0.19,  0.04,  0.26,  0.22,  0.,   -0.35,  0.06],
 [ 0.15,  0.43, -0.26, -0.45, -0.4,   0.05, -0.06,  0.34,  0.13,  0.03, -0.53, -0.04],
 [ 0.23,  0.29, -0.3,  -0.22, -0.3,  -0.14,  0.03,  0.36,  0.12,  0.03, -0.4,  -0.05],
 [ 0.56, -0.1,  -0.48,  0.09,  0.08, -0.23, -0.09, -0.02, -0.19, -0.16,  0.45, -0.07]])

Im, Ker = eliminacao_gauss(B)[0], eliminacao_gauss(B)[1]
W1, W2 = ortogonalizar(Im), ortogonalizar(Ker)

print(Im, '\n', W1)
print(Ker, '\n', W2)
print(transformacao_ortogonal(W1,W2,Im,Ker,B))


[[ 1.          0.          0.          0.          0.         -0.
   0.          0.67970152  0.46368308  0.19634297]
 [ 0.          1.          0.          0.          0.          0.
   0.          1.25982894  0.79192214 -0.75327514]
 [ 0.          0.          1.          0.          0.          0.
   0.         -0.78525923 -0.67057478 -0.54863609]
 [ 0.          0.          0.          1.          0.          0.
   0.          0.09045792 -0.27282039 -0.64545071]
 [ 0.          0.          0.          0.          1.         -0.
   0.         -1.46823413 -1.22216747  0.63802752]
 [ 0.          0.          0.          0.          0.          1.
   0.          0.27908205  0.17887438  0.43592271]
 [ 0.          0.          0.          0.          0.         -0.
   1.          1.06719228  0.98408011 -0.79418887]
 [-0.         -0.         -0.67970152 -1.25982894  0.78525923 -0.09045792
   1.46823413 -0.27908205 -1.06719228  1.        ]
 [-0.         -0.46368308 -0.79192214  0.67057478  0.272